# Notes

February 29th in 2020. I am taking that day out. because it's not in the NOAA data and 2020 had a leap year. 

# Goal

I want to automate the CAISO data. Each csv's column name should be its year and date + megawatt.
Ex: 2020-01-01Megawatts 
or something like that 

# Result

This script takes out hour-ahead forecast, solar, and wind, leaving ONLY demand. Then the demand energy is summed up by hour and put into a file. Thus this code produces hourly energy usage per file. 

In [2]:
import pandas as pd
import numpy as np
import datetime
import os
from os import listdir
from os.path import isfile, join
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [3]:
# df = pd.read_csv("/Users/yukahatori/A_Fairness/FairnessML_git/CAISO-netdemand-20200101.csv", on_bad_lines = 'skip')
# df
# '/content/drive/MyDrive/Fairness ML/FairnessML_git/Datasets/CAISO_NetDemand_Megawatts_Clean/' 
weirdOne = pd.read_csv("/content/drive/MyDrive/Fairness ML/FairnessML_git/DataProcess/CAISO-netdemand-20200311.csv", on_bad_lines = 'skip')
print("col elements when there is the weird one: ",len(weirdOne.columns))
weirdOne.drop('Unnamed: 290', axis=1, inplace= True)
weirdOne
normalOne = pd.read_csv("/content/drive/MyDrive/Fairness ML/FairnessML_git/DataProcess/CAISO-netdemand-20200101.csv", on_bad_lines = 'skip')
print("col elements when there is no weird one: ",len(normalOne.columns))

print("weird one has 5 rows, non-weird one only has 4")

print("Non-weird one")
normalOne

col elements when there is the weird one:  291
col elements when there is no weird one:  290
weird one has 5 rows, non-weird one only has 4
Non-weird one


,Net Demand 01/01/2020,00:00,00:05,00:10,00:15,00:20,00:25,00:30,00:35,00:40,...,23:15,23:20,23:25,23:30,23:35,23:40,23:45,23:50,23:55,00:00.1
0,Hour-ahead forecast,21474,21168,21168,21168,21015,21015,21015,20755,20755,...,20967,20655,20655,20655,20321,20321,20321,20078,20078,20078.0
1,Demand,21533,21429,21320,21272,21193,21109,21006,20961,20916,...,20920,20809,20681,20574,20494,20383,20297,20242,20128,20025.0
2,Solar,-33,-33,-33,-33,-33,-33,-33,-33,-33,...,-33,-33,-33,-33,-33,-33,-33,-33,-33,NaN
3,Wind,808,804,800,790,790,792,785,787,793,...,3248,3267,3307,3339,3404,3451,3482,3472,3455,NaN


In [4]:
print("Weird one")
weirdOne

Weird one


,Net Demand 03/11/2020,00:00,00:05,00:10,00:15,00:20,00:25,00:30,00:35,00:40,...,23:15,23:20,23:25,23:30,23:35,23:40,23:45,23:50,23:55,00:00.1
0,Hour-ahead forecast,21924,21656,21656,21656,21432,21432,21432,21200,21200,...,22944,22563,22563,22563,22227,22227,22227,21852,21852,21852.0
1,Demand,21809,21759,21723,21659,21501,21352,21333,21311,21219,...,22721,22600,22458,22337,22216,21916,21820,21791,21698,21578.0
2,Day-ahead net forecast,Net demand,21387,21292,21320,21248,21185,20917,20858,20885,...,21821,21680,21549,21409,21274,21129,20829,20707,20658,20563.0
3,Solar,-29,-29,-29,-29,-29,-29,-29,-29,-28,...,-27,-27,-27,-27,-27,-27,-27,-27,-27,NaN
4,Wind,506,493,470,464,468,486,495,488,469,...,1069,1083,1082,1088,1116,1126,1138,1158,1166,NaN


In [5]:
normalOne.drop(labels = [0,2, 3], axis = 0)
weirdOne.drop(labels = [0,2, 3,4], axis = 0)

,Net Demand 03/11/2020,00:00,00:05,00:10,00:15,00:20,00:25,00:30,00:35,00:40,...,23:15,23:20,23:25,23:30,23:35,23:40,23:45,23:50,23:55,00:00.1
1,Demand,21809,21759,21723,21659,21501,21352,21333,21311,21219,...,22721,22600,22458,22337,22216,21916,21820,21791,21698,21578.0


In [6]:
normalOne = normalOne.drop(labels = [0,2,3], axis = 0)
weirdOne = weirdOne.drop(labels = [0,2,3,4], axis = 0)

In [7]:
print(normalOne.columns[278:290])

def addHours(df,year, month, day):
    start = 1
    end = 14
    for hour in range(24):
        x = datetime.datetime(year, month, day, hour)
        df[x] = df.iloc[:,start:end].mean(axis = 1)
        start = end
        end +=12
    return df

Index(['23:05', '23:10', '23:15', '23:20', '23:25', '23:30', '23:35', '23:40',
       '23:45', '23:50', '23:55', '00:00.1'],
      dtype='object')


In [8]:
normalOne = addHours(normalOne, 2020, 1, 1)
weirdOne = addHours(weirdOne, 2020, 3, 11)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


In [9]:
def removeOldDates(df, firstVal):
    df = df.drop(columns = [firstVal,'00:00.1','00:00','00:05','00:10','00:15','00:20','00:25','00:30','00:35','00:40','00:45','00:50','00:55','01:00','01:05','01:10','01:15','01:20','01:25','01:30','01:35','01:40','01:45','01:50','01:55','02:00','02:05','02:10','02:15','02:20','02:25','02:30','02:35','02:40','02:45','02:50','02:55','03:00','03:05','03:10','03:15','03:20','03:25','03:30','03:35','03:40','03:45','03:50','03:55','04:00','04:05','04:10','04:15','04:20','04:25','04:30','04:35','04:40','04:45','04:50','04:55','05:00','05:05','05:10','05:15','05:20','05:25','05:30','05:35','05:40','05:45','05:50','05:55','06:00','06:05','06:10','06:15','06:20','06:25','06:30','06:35','06:40','06:45','06:50','06:55','07:00','07:05','07:10','07:15','07:20','07:25','07:30','07:35','07:40','07:45','07:50','07:55','08:00','08:05','08:10','08:15','08:20','08:25','08:30','08:35','08:40','08:45','08:50','08:55','09:00','09:05','09:10','09:15','09:20','09:25','09:30','09:35','09:40','09:45','09:50','09:55','10:00','10:05','10:10','10:15','10:20','10:25','10:30','10:35','10:40','10:45','10:50','10:55','11:00','11:05','11:10','11:15','11:20','11:25','11:30','11:35','11:40','11:45','11:50','11:55','12:00','12:05','12:10','12:15','12:20','12:25','12:30','12:35','12:40','12:45','12:50','12:55','13:00','13:05','13:10','13:15','13:20','13:25','13:30','13:35','13:40','13:45','13:50','13:55','14:00','14:05','14:10','14:15','14:20','14:25','14:30','14:35','14:40','14:45','14:50','14:55','15:00','15:05','15:10','15:15','15:20','15:25','15:30','15:35','15:40','15:45','15:50','15:55','16:00','16:05','16:10','16:15','16:20','16:25','16:30','16:35','16:40','16:45','16:50','16:55','17:00','17:05','17:10','17:15','17:20','17:25','17:30','17:35','17:40','17:45','17:50','17:55','18:00','18:05','18:10','18:15','18:20','18:25','18:30','18:35','18:40','18:45','18:50','18:55','19:00','19:05','19:10','19:15','19:20','19:25','19:30','19:35','19:40','19:45','19:50','19:55','20:00','20:05','20:10','20:15','20:20','20:25','20:30','20:35','20:40','20:45','20:50','20:55','21:00','21:05','21:10','21:15','21:20','21:25','21:30','21:35','21:40','21:45','21:50','21:55','22:00','22:05','22:10','22:15','22:20','22:25','22:30','22:35','22:40','22:45','22:50','22:55','23:00','23:05','23:10','23:15','23:20','23:25','23:30','23:35','23:40','23:45','23:50','23:55'])
    return df


In [10]:
normalOne = removeOldDates(normalOne, 'Net Demand 01/01/2020')
normalOne

,2020-01-01 00:00:00,2020-01-01 01:00:00,2020-01-01 02:00:00,2020-01-01 03:00:00,2020-01-01 04:00:00,2020-01-01 05:00:00,2020-01-01 06:00:00,2020-01-01 07:00:00,2020-01-01 08:00:00,2020-01-01 09:00:00,...,2020-01-01 14:00:00,2020-01-01 15:00:00,2020-01-01 16:00:00,2020-01-01 17:00:00,2020-01-01 18:00:00,2020-01-01 19:00:00,2020-01-01 20:00:00,2020-01-01 21:00:00,2020-01-01 22:00:00,2020-01-01 23:00:00
1,21060.461538,20250.0,19663.5,19328.666667,19184.916667,19615.25,20273.416667,20316.083333,19831.166667,19072.166667,...,18986.083333,20173.916667,21972.916667,24575.666667,24996.0,24694.75,24088.416667,23211.5,21886.583333,20557.166667


In [11]:

weirdOne = removeOldDates(weirdOne, 'Net Demand 03/11/2020')
weirdOne

,2020-03-11 00:00:00,2020-03-11 01:00:00,2020-03-11 02:00:00,2020-03-11 03:00:00,2020-03-11 04:00:00,2020-03-11 05:00:00,2020-03-11 06:00:00,2020-03-11 07:00:00,2020-03-11 08:00:00,2020-03-11 09:00:00,...,2020-03-11 14:00:00,2020-03-11 15:00:00,2020-03-11 16:00:00,2020-03-11 17:00:00,2020-03-11 18:00:00,2020-03-11 19:00:00,2020-03-11 20:00:00,2020-03-11 21:00:00,2020-03-11 22:00:00,2020-03-11 23:00:00
1,21322.333333,20528.083333,20029.416667,19783.166667,20177.0,21598.75,24063.416667,25675.666667,25335.25,24540.833333,...,21857.75,22269.833333,23053.416667,24377.416667,25762.333333,27329.916667,27026.833333,25831.166667,24013.333333,22251.25


In [12]:
normalOne = normalOne.T
normalOne

,1
2020-01-01 00:00:00,21060.461538
2020-01-01 01:00:00,20250.000000
2020-01-01 02:00:00,19663.500000
2020-01-01 03:00:00,19328.666667
2020-01-01 04:00:00,19184.916667
2020-01-01 05:00:00,19615.250000
2020-01-01 06:00:00,20273.416667
2020-01-01 07:00:00,20316.083333
2020-01-01 08:00:00,19831.166667
2020-01-01 09:00:00,19072.166667


In [13]:
weirdOne = weirdOne.T
weirdOne

,1
2020-03-11 00:00:00,21322.333333
2020-03-11 01:00:00,20528.083333
2020-03-11 02:00:00,20029.416667
2020-03-11 03:00:00,19783.166667
2020-03-11 04:00:00,20177.000000
2020-03-11 05:00:00,21598.750000
2020-03-11 06:00:00,24063.416667
2020-03-11 07:00:00,25675.666667
2020-03-11 08:00:00,25335.250000
2020-03-11 09:00:00,24540.833333


In [14]:
normalOne.rename(columns = {1: 'Megawatts'}, inplace= True)
normalOne['date'] = normalOne.index
normalOne = normalOne[['date', 'Megawatts']]

In [15]:
normalOne

,date,Megawatts
2020-01-01 00:00:00,2020-01-01 00:00:00,21060.461538
2020-01-01 01:00:00,2020-01-01 01:00:00,20250.000000
2020-01-01 02:00:00,2020-01-01 02:00:00,19663.500000
2020-01-01 03:00:00,2020-01-01 03:00:00,19328.666667
2020-01-01 04:00:00,2020-01-01 04:00:00,19184.916667
2020-01-01 05:00:00,2020-01-01 05:00:00,19615.250000
2020-01-01 06:00:00,2020-01-01 06:00:00,20273.416667
2020-01-01 07:00:00,2020-01-01 07:00:00,20316.083333
2020-01-01 08:00:00,2020-01-01 08:00:00,19831.166667
2020-01-01 09:00:00,2020-01-01 09:00:00,19072.166667


In [19]:
weirdOne.rename(columns = {1: 'Megawatts'}, inplace= True)
weirdOne['date'] = weirdOne.index
weirdOne = weirdOne[['date', 'Megawatts']]

In [22]:
normalOne.to_csv('/content/drive/MyDrive/Fairness ML/FairnessML_git/DataProcess/CAISO_test_normal.csv', encoding='utf-8', index=False)

In [21]:
weirdOne.to_csv('/content/drive/MyDrive/Fairness ML/FairnessML_git/DataProcess/CAISO_test_weird.csv', encoding='utf-8', index=False)

# Above, was an example of processing one CAISO file. Below, the process is generalized to all files

In [23]:
def processCaisoFiles(working_directory, destination_directory):
    os.chdir(working_directory)
    allDataFiles = [f for f in listdir(working_directory) if isfile(join(working_directory, f))] #returns the file names in that directory
    for fileName in allDataFiles:
        print(fileName)
        date = fileName[16:24]
        df = pd.read_csv(fileName, on_bad_lines = 'skip')
        if len(df.columns) == 290: #then it's "normal"
            df = df.drop(labels = [0,2,3], axis = 0)
        elif len(df.columns) == 291: #then it's "weird"
            df.drop('Unnamed: 290', axis=1, inplace=True)
            df = df.drop(labels = [0,2,3,4], axis = 0)
        else:
            print("something is weird.")
            return 
        # year, month, day
        #0:4 = year; 4:6 = month; 6:8 = day
        df = addHours(df, int(date[0:4]), int(date[4:6]),int(date[6:8]))
        # month/day/year
        colDate = date[4:6] +'/'+ date[6:8] +'/'+ date[0:4]
        extraCol = 'Net Demand ' + colDate
        df = removeOldDates(df, extraCol) #there's one weird column called net demand <date> that I want to remove
        df = df.T
        renameCol = 'Megawatts'
        df.rename(columns = {1: renameCol}, inplace= True)
        df['date'] = df.index
        df = df[['date', 'Megawatts']]
        newCsv = destination_directory + 'CAISO-netdemand-' + date +'-Megawatts.csv'
        df.to_csv(newCsv, encoding='utf-8', index = False)
        

In [26]:
wd = '/content/drive/MyDrive/Fairness ML/FairnessML_git/Datasets/CAISO_NetDemand_Clean'
destination = '/content/drive/MyDrive/Fairness ML/FairnessML_git/Datasets/CAISO_NetDemand_Megawatts_google/'
processCaisoFiles(wd, destination)

CAISO-netdemand-20210512.csv
CAISO-netdemand-20220119.csv
CAISO-netdemand-20200731.csv
CAISO-netdemand-20191225.csv
CAISO-netdemand-20211116.csv
CAISO-netdemand-20210118.csv
CAISO-netdemand-20200114.csv
CAISO-netdemand-20200430.csv
CAISO-netdemand-20190620.csv
CAISO-netdemand-20211129.csv
CAISO-netdemand-20200501.csv
CAISO-netdemand-20200213.csv
CAISO-netdemand-20200109.csv
CAISO-netdemand-20191024.csv
CAISO-netdemand-20190927.csv
CAISO-netdemand-20210129.csv
CAISO-netdemand-20200519.csv
CAISO-netdemand-20200630.csv
CAISO-netdemand-20210224.csv
CAISO-netdemand-20201129.csv
CAISO-netdemand-20200518.csv
CAISO-netdemand-20201026.csv
CAISO-netdemand-20191212.csv
CAISO-netdemand-20200422.csv
CAISO-netdemand-20191111.csv
CAISO-netdemand-20220207.csv
CAISO-netdemand-20210124.csv


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


CAISO-netdemand-20190105.csv
CAISO-netdemand-20201222.csv
CAISO-netdemand-20190602.csv
CAISO-netdemand-20190301.csv
CAISO-netdemand-20200606.csv
CAISO-netdemand-20211030.csv
CAISO-netdemand-20190430.csv
CAISO-netdemand-20210709.csv
CAISO-netdemand-20191020.csv
CAISO-netdemand-20200922.csv
CAISO-netdemand-20210415.csv
CAISO-netdemand-20190626.csv
CAISO-netdemand-20210315.csv
CAISO-netdemand-20190827.csv
CAISO-netdemand-20220123.csv
CAISO-netdemand-20190612.csv
CAISO-netdemand-20191107.csv
CAISO-netdemand-20200504.csv
CAISO-netdemand-20200611.csv
CAISO-netdemand-20210707.csv
CAISO-netdemand-20200205.csv
CAISO-netdemand-20191004.csv
CAISO-netdemand-20210907.csv
CAISO-netdemand-20210329.csv
CAISO-netdemand-20211024.csv
CAISO-netdemand-20191129.csv
CAISO-netdemand-20190929.csv
CAISO-netdemand-20190323.csv
CAISO-netdemand-20210605.csv
CAISO-netdemand-20210305.csv
CAISO-netdemand-20190926.csv
CAISO-netdemand-20200718.csv
CAISO-netdemand-20200703.csv
CAISO-netdemand-20210119.csv
CAISO-netdeman